In [ ]:

#@markdown * Connect to the hosted runtime and run each cell after updating the necessary inputs
#@markdown * Download the file "example_data_for_post_analysis.csv" from the folder "csv" in github.
#@markdown * Upload the csv file to your Google Drive and open it with Google Sheets
#@markdown * In the cell below, copy and paste the url of the sheet. 

In [ ]:
#@markdown ### Enter the trix id for the sheet file containing the Data: 
#@markdown The spreadsheet should contain the mandatory columns:
#@markdown * date: date in the format YYYY-MM-DD
#@markdown * geo: the number which identifies the geo
#@markdown * assignment: treatment/control assignment of the geo
#@markdown * response: variable on which you want to measure incrementality
#@markdown (e.g. sales, transactions)
#@markdown * cost: ad spend
#@markdown ---

BAZEL_VERSION = '3.0.0'
!wget https://github.com/bazelbuild/bazel/releases/download/{BAZEL_VERSION}/bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!chmod +x bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!./bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh
!sudo apt-get install python3-dev python3-setuptools git
!git clone https://github.com/google/matched_markets
!python3 -m pip install ./matched_markets

"""Loading the necessary python modules."""
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns

from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
from pandas.plotting import register_matplotlib_converters

import gspread
import warnings
from google import auth as google_auth
from google.colab import auth
from google.colab import data_table
from google.colab import widgets
from google.colab import drive
from matched_markets.methodology.common_classes import GeoAssignment
from matched_markets.methodology.utils import human_readable_number
from matched_markets.methodology.utils import plot_iroas_over_time
from matched_markets.methodology import tbr_iroas
from matched_markets.methodology.tbrdiagnostics import TBRDiagnostics

warnings.filterwarnings('ignore')

register_matplotlib_converters()
InteractiveShell.ast_node_interactivity = "all"

experiment_table = "add your url here, which should look like https://docs.google.com/spreadsheets/d/???/edit#gid=???" #@param {type:"string"}
auth.authenticate_user()
creds, _ = google_auth.default()
gc = gspread.authorize(creds)
wks = gc.open_by_url(experiment_table).sheet1
data = wks.get_all_values()
headers = data.pop(0)
data = pd.DataFrame(data, columns=headers)
data["date"] = pd.to_datetime(data["date"])
for colname in ["geo", "assignment", "response", "cost"]:
  data[colname] = pd.to_numeric(data[colname])

In [ ]:
#@title Summary of the data for the design, test, and test+cooldown period 

pretest_start_date = "2015-01-05" #@param {type:"date"}
pretest_end_date = "2015-02-15" #@param {type:"date"}
test_start_date = "2015-02-16" #@param {type:"date"}
test_end_date = "2015-03-15" #@param {type:"date"}
cooldown_end_date = "2015-04-07" #@param {type:"date"}

#@markdown Use an average order value of 1 if the experiment is based on
#@markdown sales/revenue or an actual average order value (e.g. 80$) for an
#@markdown experiment based on transactions/footfall/contracts.
average_order_value = 1 #@param{type: "number"}

test_start_date = pd.to_datetime(test_start_date)
test_end_date = pd.to_datetime(test_end_date)
cooldown_end_date = pd.to_datetime(cooldown_end_date)
pretest_start_date = pd.to_datetime(pretest_start_date)
pretest_end_date = pd.to_datetime(pretest_end_date)

# these are numerical identifier used in the table in input to identify the two
# groups
group_treatment = GeoAssignment.TREATMENT
group_control = GeoAssignment.CONTROL

dates_in_input = [pretest_start_date, pretest_end_date, test_start_date, test_end_date, cooldown_end_date]
if sorted(dates_in_input) != dates_in_input:
  raise ValueError((f'{Fore.RED}ERROR: the dates in input ' +
                    f'[pretest_start_date, pretest_end_date, test_start_date, '
                    f'test_end_date, cooldown_end_date]={dates_in_input} '
                    f'are not valid as not in chronological order. ' +
                    f'Please correct them.\n{Style.RESET_ALL}'))

geox_data = data.copy()
geox_data = geox_data[geox_data["date"]>=pretest_start_date]
geox_data["period"] = geox_data["date"].apply(
    lambda row: 0 if row in pd.Interval(
        pretest_start_date, pretest_end_date, closed="both") else
    (1 if row in pd.Interval(test_start_date, test_end_date, closed="both") else
     (2 if row in pd.Interval(test_end_date, cooldown_end_date, closed="right")
      else -1)))

total_cost = geox_data.loc[geox_data["period"]==1, "cost"].sum()
print("Total cost: {}".format(human_readable_number(total_cost)))

treatment_cost = geox_data.loc[(geox_data["period"] == 1) &
                               (geox_data["assignment"] == group_treatment),
                               "cost"].sum()
print("Treatment cost: {}".format(human_readable_number(treatment_cost)))

control_cost = geox_data.loc[(geox_data["period"] == 1) &
                             (geox_data["assignment"] == group_control),
                             "cost"].sum()
print("Control cost: {}".format(human_readable_number(control_cost)))

print("Total response and cost by period and group")
output_table = geox_data.loc[
    geox_data["period"].isin([0, 1]),
    ["period", "assignment", "response", "cost"]].groupby(
        ["period", "assignment"], as_index=False).sum()
output_table.assignment = output_table.assignment.map(
    {group_control: "Control", group_treatment: "Treatment"})
output_table.period = output_table.period.map({0: "Pretest", 1: "Test"})

data_table.DataTable(output_table, include_index=False)

# identify outlier dates
diag = TBRDiagnostics()
diag.fit(geox_data, key_group="assignment", group_control=group_control,
                    group_treatment=group_treatment)
outlier_dates = sorted(diag.get_test_results()["outlier_dates"])
# remove outlier dates from data
geox_data = geox_data[~geox_data["date"].isin(outlier_dates)]
if outlier_dates:
  print(f'\nThe following days have been removed as they have been identified' +
        f' as outliers: {[x.strftime("%Y-%m-%d") for x in outlier_dates]}')
  updated_cost = geox_data.loc[geox_data["period"]==1, "cost"].sum()
  print(f'Total cost excluding outlier dates: ' +
        f'{human_readable_number(updated_cost)}')

In [ ]:
#@title Visualization of experiment data. 

# set basic plot parameters
chart_width = 600
chart_height = 300
axis_title_font_size = 16
axis_label_font_size = 14
title_font_size = 19

plot_dict = {"Response": {"colname": "response", "format": "s",
                          "attr": "tbr_response"},
             "Ad Spend": {"colname": "cost", "format": "s", "attr": "tbr_cost"}}

experiment_dates = pd.DataFrame({
    "date": [
        test_start_date, test_end_date, pretest_start_date, pretest_end_date
    ],
    "color": [
        "Experiment period", "Experiment period", "Pretest period",
        "Pretest period"
    ]
})

cooldown_date = pd.DataFrame({"date": [cooldown_end_date],
                              "color": ["End of cooldown period"] })
tb = widgets.TabBar(list(plot_dict.keys()))
for k, v in plot_dict.items():
  with tb.output_to(k):
    # Set y-axis format
    format_value = ".1" if v["format"] == "s" else ".2"
    format_string = format_value + v["format"]

    # Set dataframe to use for plotting and define y-variable to plot
    plot_df = geox_data[geox_data["assignment"].isin(
        [group_control,
        group_treatment])].groupby(["date", "assignment"],
                                    as_index=False)[["response", "cost"]].sum()
    plot_df["assignment"] = plot_df["assignment"].map({
        group_control: "Control",
        group_treatment: "Treatment"
    })
    y_string = v["colname"] + ":Q"

    # Define how we select based on where the mouse is.
    selection = alt.selection_single(
        fields=["date"],
        nearest=True,
        on="mouseover",
        empty="none",
        clear="mouseout")
    bottom_selection = alt.selection_single(
        fields=["date"],
        nearest=True,
        on="mouseover",
        empty="none",
        clear="mouseout")

    # Brush for selecting a location to zoom into on x-axis.
    brush = alt.selection(type="interval", encodings=["x"])

    # Base chart -- same for top and bottom
    base = alt.Chart(plot_df).mark_line().encode(
        x=alt.X("date:T", axis=alt.Axis(title="", format=("%b %e")))
    )

    # Lines for data
    lines = base.mark_line().encode(
        y=alt.Y(y_string, axis=alt.Axis(title=" ", format=(format_string)))
    )
    bottom_lines = lines.encode(alt.X("date:T", scale=alt.Scale(domain=brush)))

    # Vertical rule
    rule = base.mark_rule().encode(
        opacity=alt.condition(selection, alt.value(0.3), alt.value(0)),
        tooltip=["date:T", y_string]
    ).add_selection(selection)
    bottom_rule = base.mark_rule().encode(
        opacity=alt.condition(bottom_selection, alt.value(0.3), alt.value(0)),
        tooltip=["date:T", y_string]
    ).add_selection(bottom_selection)

    #
    date_rule = alt.Chart(experiment_dates).mark_rule(strokeWidth=2).encode(
        x='date:T',
        color=alt.Color(
            'color',
            scale=alt.Scale(
                domain=[
                    'Experiment period', 'Pretest period',
                    'End of cooldown period', 'Control', 'Treatment'
                ],
                range=['black', 'red', 'black', '#1f77b4', '#ff7f0e'])))

    cooldown_date_rule = alt.Chart(cooldown_date).mark_rule(
        strokeWidth=2, strokeDash=[5, 2], color='black').encode(
            x='date:T', color='color:N')

    # Points to denote where the mouse is.
    points = lines.mark_point().transform_filter(selection)
    bottom_points = bottom_lines.mark_point().transform_filter(bottom_selection)

    lines = lines.mark_line().encode(
      color=alt.Color("assignment", title=" "))
    bottom_lines = bottom_lines.mark_line().encode(
      color=alt.Color("assignment", title=" "))
    base_rule = base.transform_pivot(
        "assignment", value=v["colname"],
        groupby=["date"]).mark_rule().encode(tooltip=["date:T"] + [
            alt.Tooltip(c, type="quantitative")
            for c in plot_df["assignment"].unique()
        ])
    rule = base_rule.encode(
        opacity=alt.condition(selection, alt.value(0.3), alt.value(0))
    ).add_selection(selection)
    bottom_rule = base_rule.encode(
        opacity=alt.condition(bottom_selection, alt.value(0.3), alt.value(0))
    ).add_selection(bottom_selection)

    # Compile top chart
    top_chart = alt.layer(
        lines,
        points,
        rule,
        date_rule,
        cooldown_date_rule
    ).add_selection(
        brush
    ).properties(
        width=chart_width,
        height=chart_height,
        title=k
    )

    # Compile bottom chart
    bottom_chart = alt.layer(
        bottom_lines,
        bottom_rule,
        bottom_points
    ).properties(
        width=chart_width,
        height=chart_height
    )

    # Combine charts
    final_chart = alt.vconcat(
        top_chart,
        bottom_chart
    )
    final_chart.configure_axis(
        titleFontSize=axis_title_font_size,
        labelFontSize=axis_label_font_size
    ).configure_title(
        fontSize=title_font_size
    ).display()



In [ ]:
#@title Results of the experiment. 
tb = widgets.TabBar(['Results without cooldown period', 'Results with cooldown period'])
use_cooldown = [False, True]
period_to_use = [[1], [1, 2]]

for ind in range(2):
  with tb.output_to(ind):
    tbr_results = tbr_iroas.TBRiROAS(use_cooldown=use_cooldown[ind])
    tbr_results.fit(geox_data, key_group='assignment',
                    group_control=group_control,
                    group_treatment=group_treatment)
    results = tbr_results.summary(level=0.8, tails=2)

    print('Summary of the results of TBR:\n')
    print('estimate\t precision\t ci_level\t lower bound\t upper bound')
    print(
        '{:.3f}\t\t {:.3f}\t\t {:.2f}\t\t {:.2f}\t\t {:.2f}\n\n'.format(
            results.estimate.values[0] * average_order_value,
            results.precision.values[0] * average_order_value,
            results.level.values[0],
            results.lower.values[0] * average_order_value,
            results.upper.values[0] * average_order_value))

    print(f'Probability that the iROAS is >= ' +
          f'{results.posterior_threshold.values[0]}:' +
          f' {results.probability.values[0]}')

    print('\nincremental cost = {}'.format(
        human_readable_number(results.incremental_cost.values[0])))
    print('\nincremental response = {}'.format(
        human_readable_number(results.incremental_response.values[0])))
    print('\nincremental response as % of treatment response = {:.2f}%\n'.format(
          results.relative_lift.values[0] * 100))

In [ ]:
# @title Visualization of the results. 

# set basic plot parameters
chart_width = 750
chart_height = 300
axis_title_font_size = 16
axis_label_font_size = 14
title_font_size = 19

level = 0.8
tails = 2

experiment_dates = pd.DataFrame({
    "date": [
        test_start_date, test_end_date, pretest_start_date, pretest_end_date
    ],
    "color": [
        "Experiment period", "Experiment period", "Pretest period",
        "Pretest period"
    ]
})

cooldown_date = pd.DataFrame({"date": [cooldown_end_date],
                              "color": ["End of cooldown period"] })

periods = (tbr_results.periods.pre, tbr_results.periods.test,
           tbr_results.periods.cooldown)

nsims = 10000
alpha = (1 - level)/tails

if tbr_results._is_fixed_cost_scenario():
  plot_dict = {"Response": {"colname": "response", "format": "s",
                            "attr": "tbr_response"}}

tb = widgets.TabBar(list(plot_dict.keys()) + ["iROAS"])
for k, v in plot_dict.items():
  with tb.output_to(k):
    # Obtain the distributions of the two sets of causal effects
    delta_response = getattr(tbr_results,
                            v['attr']).causal_cumulative_distribution()
    pointwise_difference = getattr(tbr_results, v["attr"]).causal_effect(
        periods).reset_index().rename(columns={0: "response"})
    lower = np.diff(delta_response.ppf(alpha), prepend=0)
    lower = np.concatenate(
        (pointwise_difference.loc[pointwise_difference["date"] < test_start_date,
                                  "response"].values, lower))
    upper = np.diff(delta_response.ppf(1 - alpha), prepend=0)
    upper = np.concatenate(
        (pointwise_difference.loc[pointwise_difference["date"] < test_start_date,
                                  "response"].values, upper))
    ci_bands = pd.DataFrame({
        "date": geox_data.loc[geox_data["period"]>=0, "date"].unique(),
        "lower": np.round(lower, 2),
        "upper": np.round(upper, 2),
        "pointwise_difference": np.round(pointwise_difference["response"], 2)
    })

    cumul_effect = np.cumsum(
        getattr(tbr_results,
                v["attr"]).causal_effect(periods)).reset_index().rename(
                    columns={0: "response"})
    cumul_effect = cumul_effect.loc[
        cumul_effect["date"].between(test_start_date, cooldown_end_date),
        "response"]
    cumulative_df = pd.DataFrame({
        "date": geox_data.loc[geox_data["period"]>=1, "date"].unique(),
        "lower": np.round(delta_response.ppf(alpha), 2),
        "upper": np.round(delta_response.ppf(1 - alpha), 2),
        "cumulative_effect": np.round(cumul_effect, 2)
    })


    # Set y-axis format
    format_string = ".1s"

    # Set dataframe to use for plotting and define y-variable to plot
    tmp_data = geox_data[geox_data["assignment"] == group_treatment].groupby(
        ["date", "assignment"],
        as_index=False)[["response", "cost"]].sum()

    period_index = getattr(tbr_results, v["attr"])._make_period_index(periods)
    # Get the test- period data in the form needed for regression.
    treat_vec = getattr(tbr_results, v["attr"])._response_vector(period_index)
    cntrl_mat = getattr(tbr_results, v["attr"])._design_matrix(period_index)
    # Calculate the causal effect of the campaign.
    counterfactual = round(getattr(tbr_results, v["attr"]).predict(cntrl_mat), 2)
    counterfactual = counterfactual.reset_index().rename(
        columns={0: v['colname']})
    # for the counterfactual we used the same identifier as control.
    counterfactual["assignment"] = group_control

    plot_df = pd.concat([tmp_data, counterfactual], sort=False)
    plot_df["assignment"] = plot_df["assignment"].map({
        group_control: "Counterfactual",
        group_treatment: "Observed"
    })
    y_string = v["colname"] + ":Q"

    # Define how we select based on where the mouse is.
    selection = alt.selection_single(
      fields=["date"], nearest=True, on="mouseover", empty="none",
      clear="mouseout")
    middle_selection = alt.selection_single(
      fields=["date"], nearest=True, on="mouseover", empty="none",
      clear="mouseout")
    bottom_selection = alt.selection_single(
      fields=["date"], nearest=True, on="mouseover", empty="none",
      clear="mouseout")


    # Brush for selecting a location to zoom into on x-axis.
    brush = alt.selection(type="interval", encodings=["x"])

    # Base chart
    base = alt.Chart(plot_df).mark_line().encode(
        x=alt.X("date:T", axis=alt.Axis(title="", format=("%b %e")))
    )
    middle_base = alt.Chart(ci_bands).mark_line().encode(
        x=alt.X("date:T", axis=alt.Axis(title="", format=("%b %e")))
    )
    bottom_base = alt.Chart(cumulative_df).mark_line().encode(
        x=alt.X("date:T", axis=alt.Axis(title="", format=("%b %e")))
    )
    # Lines for data
    lines = base.mark_line().encode(
        y=alt.Y(y_string, axis=alt.Axis(title=" ", format=(format_string)))
    )
    middle_lines = middle_base.mark_line().encode(
        y=alt.Y(
            "pointwise_difference:Q",
            axis=alt.Axis(title=" ", format=(format_string))))
    bottom_lines = bottom_base.mark_line().encode(
        y=alt.
        Y("cumulative_effect:Q", axis=alt.Axis(title=" ", format=(
            format_string))))
    # Vertical rule
    rule = base.mark_rule().encode(
        opacity=alt.condition(selection, alt.value(0.3), alt.value(0)),
        tooltip=["date:T", y_string]
    ).add_selection(selection)
    middle_rule = middle_base.mark_rule().encode(
        opacity=alt.condition(middle_selection, alt.value(0.3), alt.value(0)),
        tooltip=["date:T", "pointwise_difference:Q"]
    ).add_selection(middle_selection)
    bottom_rule = bottom_base.mark_rule().encode(
        opacity=alt.condition(middle_selection, alt.value(0.3), alt.value(0)),
        tooltip=["date:T", "cumulative_effect:Q"]
    ).add_selection(middle_selection)
    #
    date_rule = alt.Chart(experiment_dates).mark_rule(strokeWidth=2).encode(
        x='date:T',
        color=alt.Color(
            'color',
            scale=alt.Scale(
                domain=[
                    'Experiment period', 'Pretest period',
                    'End of cooldown period', 'Counterfactual', 'Observed'
                ],
                range=['black', 'red', 'black', '#1f77b4', '#ff7f0e'])))

    cooldown_date_rule = alt.Chart(cooldown_date).mark_rule(
        strokeWidth=2,
        strokeDash=[5,2],
        color='black'
    ).encode(
        x='date:T',
        color='color:N'
    )

    # Points to denote where the mouse is.
    points = lines.mark_point().transform_filter(selection)
    middle_points = middle_lines.mark_point().transform_filter(middle_selection)
    bottom_points = bottom_lines.mark_point().transform_filter(bottom_selection)

    lines = lines.mark_line().encode(
      color=alt.Color("assignment", title=" "))
    middle_lines = middle_lines.mark_line(color="blue").encode()
    bottom_lines = bottom_lines.mark_line(color="blue").encode()
    base_rule = base.transform_pivot(
        "assignment", value=v["colname"],
        groupby=["date"]).mark_rule().encode(tooltip=["date:T"] + [
            alt.Tooltip(c, type="quantitative")
            for c in plot_df["assignment"].unique()
        ])
    base_rule1 = middle_base.mark_rule().encode(
        tooltip=["date:T", "pointwise_difference:Q", "lower:Q", "upper:Q"])
    base_rule2 = bottom_base.mark_rule().encode(
        tooltip=["date:T", "cumulative_effect:Q", "lower:Q", "upper:Q"])
    rule = base_rule.encode(
        opacity=alt.condition(selection, alt.value(0.3), alt.value(0))
    ).add_selection(selection)
    middle_rule = base_rule1.encode(
        opacity=alt.condition(middle_selection, alt.value(0.3), alt.value(0))
    ).add_selection(middle_selection)
    bottom_rule = base_rule2.encode(
        opacity=alt.condition(bottom_selection, alt.value(0.3), alt.value(0))
    ).add_selection(bottom_selection)

    middle_ci_bands_rule = alt.Chart(ci_bands).mark_area(color="gray").encode(
      alt.X("date:T", scale=alt.Scale(domain=brush)),
      y='lower:Q',
      y2='upper:Q',
      opacity=alt.value(0.5)
    )
    bottom_ci_bands_rule = alt.Chart(cumulative_df).mark_area(
        color='gray').encode(
            alt.X('date:T', scale=alt.Scale(domain=brush)),
            y='lower:Q',
            y2='upper:Q',
            opacity=alt.value(0.5))
    # Compile top chart
    top_chart = alt.layer(
        lines,
        points,
        rule,
        date_rule,
        cooldown_date_rule
    ).add_selection(
        brush
    ).properties(
        width=chart_width,
        height=chart_height,
        title="Observed vs. Counterfactual"
    )

    # Compile middle chart
    middle_chart = alt.layer(
        middle_lines,
        middle_rule,
        middle_points,
        date_rule,
        cooldown_date_rule,
        middle_ci_bands_rule
    ).properties(
        width=chart_width,
        height=chart_height,
        title="Pointwise difference"
    )

    # Compile bottom chart
    bottom_chart = alt.layer(
        bottom_lines,
        bottom_rule,
        bottom_points,
        date_rule,
        cooldown_date_rule,
        bottom_ci_bands_rule
    ).properties(
        width=chart_width,
        height=chart_height,
        title="Cumulative effect"
    )
    # Combine charts
    final_chart = alt.vconcat(
        top_chart,
        middle_chart,
        bottom_chart
    )
    final_chart.configure_axis(
        titleFontSize=axis_title_font_size,
        labelFontSize=axis_label_font_size
    ).configure_title(
        fontSize=title_font_size
    ).display()

if tbr_results._is_fixed_cost_scenario():
  iroas_dist = tbr_results.tbr_response.causal_cumulative_distribution()
  cost = np.cumsum(
      tbr_results.tbr_cost.causal_effect(
          (tbr_results.periods.test, tbr_results.periods.cooldown)))
  iroas_df = pd.DataFrame({
      "date": geox_data.loc[geox_data["period"]>=1, "date"].unique(),
      "lower": np.round(iroas_dist.ppf(alpha) / cost, 2),
      "upper": np.round(iroas_dist.ppf(1 - alpha) / cost, 2),
      "mean": np.round(iroas_dist.mean() / cost, 2)
  })

else:
  # Obtain the distributions of the two sets of causal effects
  delta_response = tbr_results.tbr_response.causal_cumulative_distribution()
  # We know that causal costs only arose during the test period.
  delta_cost = tbr_results.tbr_cost.causal_cumulative_distribution(
      periods=(tbr_results.periods.test, tbr_results.periods.cooldown))

  # Simulate the iROAS
  sims_cost = delta_cost.rvs([nsims, len(delta_cost.mean())])
  sims_response = delta_response.rvs([nsims, len(delta_response.mean())])
  sims_iroas = sims_response / sims_cost
  iroas_df = pd.DataFrame({
    "date": pd.date_range(test_start_date, cooldown_end_date, freq="D"),
    "lower": np.round(np.percentile(sims_iroas, 100 * alpha, axis=0), 2),
    "upper": np.round(np.percentile(sims_iroas, 100 * (1 - alpha), axis=0), 2),
    "mean": np.round(np.mean(sims_iroas, axis=0), 2)
  })

final_chart = plot_iroas_over_time(iroas_df, experiment_dates, cooldown_date)

with tb.output_to("iROAS"):
  final_chart.properties(
      width=chart_width,
      height=chart_height,
      title="iROAS estimate over time"
  ).configure_axis(
      titleFontSize=axis_title_font_size,
      labelFontSize=axis_label_font_size
  ).configure_title(
      fontSize=title_font_size
  ).display()